In [ ]:
from astropy.io import fits
from io import BytesIO
import sys
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt
sys.path.append('/home/bekah/gPhoton2')
from gPhoton.coadd import zero_flag_and_edge
import pyarrow.parquet as parquet

In [ ]:
from quickbin import bin2d
import pyarrow.parquet as parquet
import pandas as pd 

# photon table would already exist in gphoton2 
photons = parquet.read_table("/home/bekah/Downloads/e00848-fd-b00.parquet").to_pandas()

# want photons that are on detector, no flags, has aspect soln 
photons = photons[(photons['detrad'] < 600)]
photons = photons.dropna(subset=['ra', 'dec'])
photons = photons[(photons['flags'] < 1)]

# bin2d for ra & dec std + count, also get vals for q range 
ra = bin2d(photons['col'],photons['row'],photons['ra'],n_bins=800,op=('count', 'std'))
dec = bin2d(photons['col'],photons['row'],photons['dec'],n_bins=800,op=('std'))
q_m =  bin2d(photons['col'],photons['row'],photons['q'],n_bins=800,op=('min', 'max'))

# dispersion metric 
disp = ra['std']+dec # bin2d only returns a dictionary when more than one stat is calculated 

# rough exp time (use the correct one for threshold calc)
time = abs(photons['t'].iloc[0]-photons['t'].iloc[-1])


In [ ]:
# getting threshold 
high_disp = np.argwhere(disp>.016)
values = ra['count'][high_disp[:, 0], high_disp[:, 1]]
max(values/time)*.5

In [ ]:
plt.hist(values/time)

In [ ]:
# fig 1: DISPERSION 

plt.scatter(disp,ra['count']/time,s=.1)
plt.ylabel("total photons per bin div by expt")
plt.xlabel('dispersion per bin')

In [ ]:
# fig 2: Q (PULSE HEIGHT) RANGE

plt.scatter(q_m['max']-q_m['min'],ra['count']/time,s=.1,marker='.')
plt.ylabel("total photons per bin div by expt")
plt.xlabel('range of pulse height per bin')